# Agent-Based Model of Evans 1997 Migraine CEA using Mesa 3

HOME: <https://di4health.github.io>

NOTE: This notebook was created by Antropic's Claude Code. Here was my workflow:

1. I gave Claude Code Julia's `Agents.jl` agent-based model. It was a straightforward translation. Claude provide one warning:

> For Python, "1M agents may be slow — Python is ~10–50× slower than Julia for this loop-heavy pattern. If it takes too long, reduce N to 100,000 initially to verify correctness, then scale up."

The notebook below is unedited for your review. Your feedback and questions are welcome. 

---

Tomás Aragón

This notebook replicates the Evans 1997 migraine cost-effectiveness analysis as an agent-based microsimulation using Python's [Mesa 3](https://github.com/projectmesa/mesa) framework. It parallels the Julia `Agents.jl` analysis.

| Approach | Language | Method | Output |
|----------|----------|--------|--------|
| `DecisionProgramming.jl` | Julia | Analytical (influence diagram, MILP) | Exact E[Cost], E[Utility] |
| `pyAgrum` | Python | Analytical (influence diagram, Shafer-Shenoy) | Exact E[Cost], E[Utility] |
| `Agents.jl` | Julia | Simulation (ABM/microsimulation) | Monte Carlo estimates ± SE |
| **`Mesa 3`** (this notebook) | **Python** | **Simulation (ABM/microsimulation)** | **Monte Carlo estimates ± SE** |

Sources:
- Evans, K. W., et al. *PharmacoEconomics* 12(5): 565–77 (1997).
- Briggs, A. H., K. Claxton, and M. Sculpher. *Decision Modelling for Health Economic Evaluation.* Oxford, 2011.
- ter Hoeven, E., et al. "Mesa 3: Agent-based modeling with Python in 2025." *JOSS* 10(107): 7668 (2025).

In [10]:
import mesa
import numpy as np
import pandas as pd

print(f"Mesa version: {mesa.__version__}")

Mesa version: 3.4.2


## 1. Agent definition

Each agent is a migraine patient who records treatment arm, accumulated cost, utility outcome, and terminal leaf (A–J). No spatial interactions — patients traverse the decision tree independently.

In [11]:
class Patient(mesa.Agent):
    """A migraine patient who traverses the Evans 1997 decision tree."""

    def __init__(self, model, treatment):
        super().__init__(model)
        self.treatment = treatment   # "sumatriptan" or "caffeine"
        self.cost = 0.0
        self.utility = 0.0
        self.leaf = "unresolved"     # terminal node A–J

    def step(self):
        """Traverse the entire decision tree in one step."""
        if self.leaf != "unresolved":
            return  # already resolved

        p = self.model.params
        rng = self.model.random

        # Treatment-specific parameters
        is_suma = self.treatment == "sumatriptan"
        p_relief       = p["p_relief_suma"]       if is_suma else p["p_relief_caff"]
        p_norecurrence = p["p_norecurrence_suma"] if is_suma else p["p_norecurrence_caff"]
        c_drug         = p["c_sumatriptan"]       if is_suma else p["c_caffeine"]

        self.cost = c_drug                         # initial drug cost

        if rng.random() < p_relief:                # ── Relief? ──
            if rng.random() < p_norecurrence:      #   No recurrence → A/F
                self.utility = p["u_relief_norec"]
                self.leaf = "A" if is_suma else "F"
            else:                                  #   Recurrence → B/G (+2nd dose)
                self.cost += c_drug
                self.utility = p["u_relief_rec"]
                self.leaf = "B" if is_suma else "G"
        else:                                      # ── No relief ──
            if rng.random() < p["p_endures"]:      #   Endures → C/H
                self.utility = p["u_norelief_endures"]
                self.leaf = "C" if is_suma else "H"
            else:                                  #   ED visit
                self.cost += p["c_ed"]
                if rng.random() < p["p_relief_ed"]:#     ED relief → D/I
                    self.utility = p["u_norelief_ed"]
                    self.leaf = "D" if is_suma else "I"
                else:                              #     Hospital → E/J
                    self.cost += p["c_hospital"]
                    self.utility = p["u_norelief_endures"]
                    self.leaf = "E" if is_suma else "J"

## 2. Decision tree (visual reference)

```
Treatment ─┬─ Relief? ── Yes ─┬─ Recurrence? ── No  → A/F (u=1.0)
           │                  └──────────────── Yes → B/G (u=0.9, +drug cost)
           └─ Relief? ── No  ─┬─ Endures?  ─── Yes → C/H (u=−0.3)
                              └─ ED visit  ───┬─ Relief   → D/I (u=0.1)
                                              └─ Hospital → E/J (u=−0.3)
```

## 3. Model

Both treatment arms in a single model. All 14 parameters stored as model properties for sensitivity analysis.

In [12]:
class EvansMigraineABM(mesa.Model):
    """Evans 1997 migraine CEA microsimulation."""

    def __init__(self, n_per_arm=1_000_000, seed=42):
        super().__init__(seed=seed)

        # Evans 1997 parameters (4 costs, 4 utilities, 6 probabilities)
        self.params = {
            # Costs ($Can)
            "c_sumatriptan": 16.10,
            "c_caffeine":    1.32,
            "c_ed":          63.16,
            "c_hospital":    1093.0,
            # Utilities
            "u_relief_norec":     1.0,
            "u_relief_rec":       0.9,
            "u_norelief_endures": -0.30,
            "u_norelief_ed":      0.1,
            # Probabilities
            "p_relief_suma":       0.558,
            "p_relief_caff":       0.379,
            "p_norecurrence_suma": 0.594,
            "p_norecurrence_caff": 0.703,
            "p_endures":           0.92,
            "p_relief_ed":         0.998,
        }

        # Create agents — both arms interleaved
        for _ in range(n_per_arm):
            Patient(self, treatment="sumatriptan")
            Patient(self, treatment="caffeine")

    def step(self):
        self.agents.do("step")  # sequential (order doesn't matter — no interaction)

## 4. Run the simulation

In [13]:
N = 1_000_000
model = EvansMigraineABM(n_per_arm=N, seed=2026)
model.step()                                    # one step resolves all agents
print(f"Simulated {len(model.agents)} patients ({N} per arm)")

Simulated 2000000 patients (1000000 per arm)


## 5. Results

In [14]:
# Collect agent data into DataFrame
df = pd.DataFrame({
    "treatment": [a.treatment for a in model.agents],
    "cost":      [a.cost      for a in model.agents],
    "utility":   [a.utility   for a in model.agents],
    "leaf":      [a.leaf      for a in model.agents],
})

# Split by arm
d1 = df[df.treatment == "sumatriptan"]
d2 = df[df.treatment == "caffeine"]

EC_D1, EU_D1 = d1.cost.mean(), d1.utility.mean()
EC_D2, EU_D2 = d2.cost.mean(), d2.utility.mean()
dC = EC_D1 - EC_D2
dU = EU_D1 - EU_D2
ICER = (dC / dU) * 365                          # annualize 24h → 1 year

print(f"=== Cost-Effectiveness Analysis (N = {N:,} per arm) ===")
print(f"{'Strategy':<20} {'E[Cost]':>10} {'E[Utility]':>12}")
print(f"{'Caffeine/Ergot':<20} {EC_D2:10.4f} {EU_D2:12.4f}")
print(f"{'Sumatriptan':<20} {EC_D1:10.4f} {EU_D1:12.4f}")
print()
print(f"Incremental Cost:    {dC:10.4f}")
print(f"Incremental Utility: {dU:10.4f}")
print(f"ICER: ${ICER:.0f} Can/QALY")

=== Cost-Effectiveness Analysis (N = 1,000,000 per arm) ===
Strategy                E[Cost]   E[Utility]
Caffeine/Ergot           4.6962       0.2019
Sumatriptan             22.0360       0.4171

Incremental Cost:       17.3398
Incremental Utility:     0.2151
ICER: $29423 Can/QALY


### Comparison with analytical (exact) reference values

In [15]:
# Analytical reference from DecisionProgramming.jl / R rdecision / pyAgrum
ref = {"EC_D1": 22.058057, "EU_D1": 0.4168609,
       "EC_D2": 4.714972,  "EU_D2": 0.2012760, "ICER": 29363.0}

print("=== ABM vs Analytical ===")
print(f"{'':24} {'ABM':>10} {'Exact':>10} {'Diff':>10}")
print(f"{'E[C|Sumatriptan]':<24} {EC_D1:10.4f} {ref['EC_D1']:10.4f} {EC_D1 - ref['EC_D1']:+10.4f}")
print(f"{'E[U|Sumatriptan]':<24} {EU_D1:10.4f} {ref['EU_D1']:10.4f} {EU_D1 - ref['EU_D1']:+10.6f}")
print(f"{'E[C|Caffeine]':<24} {EC_D2:10.4f} {ref['EC_D2']:10.4f} {EC_D2 - ref['EC_D2']:+10.4f}")
print(f"{'E[U|Caffeine]':<24} {EU_D2:10.4f} {ref['EU_D2']:10.4f} {EU_D2 - ref['EU_D2']:+10.6f}")
print(f"{'ICER':<24} {ICER:10.0f} {ref['ICER']:10.0f} {ICER - ref['ICER']:+10.0f}")

=== ABM vs Analytical ===
                                ABM      Exact       Diff
E[C|Sumatriptan]            22.0360    22.0581    -0.0220
E[U|Sumatriptan]             0.4171     0.4169  +0.000192
E[C|Caffeine]                4.6962     4.7150    -0.0187
E[U|Caffeine]                0.2019     0.2013  +0.000673
ICER                          29423      29363        +60


### Path frequencies

In [16]:
# Analytical path probabilities
p_exact = {
    "A": 0.331452, "B": 0.226548, "C": 0.406640, "D": 0.035289, "E": 0.000071,
    "F": 0.266437, "G": 0.112563, "H": 0.571320, "I": 0.049581, "J": 0.000099,
}

freq = (df.groupby(["treatment", "leaf"])
          .size()
          .reset_index(name="n"))
freq["simulated"]  = freq["n"] / N
freq["analytical"] = freq["leaf"].map(p_exact)
freq["diff"]       = freq["simulated"] - freq["analytical"]
freq.drop(columns="n")

,treatment,leaf,simulated,analytical,diff
0,caffeine,F,0.266914,0.266437,0.000477
1,caffeine,G,0.112742,0.112563,0.000179
2,caffeine,H,0.571062,0.571320,-0.000258
3,caffeine,I,0.049177,0.049581,-0.000404
4,caffeine,J,0.000105,0.000099,0.000006
5,sumatriptan,A,0.332125,0.331452,0.000673
6,sumatriptan,B,0.226102,0.226548,-0.000446
7,sumatriptan,C,0.406774,0.406640,0.000134
8,sumatriptan,D,0.034921,0.035289,-0.000368
9,sumatriptan,E,0.000078,0.000071,0.000007


### Monte Carlo standard errors and 95% confidence intervals

In [17]:
def se(x):
    return x.std() / np.sqrt(len(x))

print("=== 95% Confidence Intervals ===")
print(f"E[C|Sumatriptan]: {EC_D1:8.4f} ± {1.96 * se(d1.cost):.4f}")
print(f"E[U|Sumatriptan]: {EU_D1:8.4f} ± {1.96 * se(d1.utility):.6f}")
print(f"E[C|Caffeine]:    {EC_D2:8.4f} ± {1.96 * se(d2.cost):.4f}")
print(f"E[U|Caffeine]:    {EU_D2:8.4f} ± {1.96 * se(d2.utility):.6f}")

=== 95% Confidence Intervals ===
E[C|Sumatriptan]:  22.0360 ± 0.0320
E[U|Sumatriptan]:   0.4171 ± 0.001206
E[C|Caffeine]:      4.6962 ± 0.0354
E[U|Caffeine]:      0.2019 ± 0.001191


## Summary

With 1M agents per arm the ABM converges to the analytical ICER ≈ \$29,363 Can/QALY.

**Advantages of the ABM over the analytical solution:**
- Full **distribution** of individual outcomes (not just expected values)
- **Confidence intervals** directly from Monte Carlo standard errors
- Natural extension to **patient heterogeneity** (age-varying probabilities, comorbidities)
- Straightforward to add **multi-period dynamics** (repeated episodes) or **resource constraints** (ED capacity)

**Trade-off:** Analytical methods (DecisionProgramming.jl, pyAgrum) give exact answers instantly; the ABM requires many agents for precision but produces richer output.

### Julia Agents.jl vs Python Mesa 3

| Feature | Agents.jl | Mesa 3 |
|---------|-----------|--------|
| Agent definition | `@agent struct Patient(NoSpaceAgent)` | `class Patient(mesa.Agent)` |
| Model creation | `StandardABM(Patient; agent_step!, ...)` | `class Model(mesa.Model)` with `step()` |
| Add agents | `add_agent!(model; treatment=...)` | `Patient(model, treatment=...)` |
| Step | `step!(model, 1)` | `model.step()` |
| Agent activation | `agent_step!` callback | `model.agents.do("step")` |
| RNG | `abmrng(model)` → `rand(rng)` | `self.model.random` → `rng.random()` |
| Properties | `abmproperties(model)` (NamedTuple) | `self.model.params` (dict) |
| All agents | `allagents(model)` | `model.agents` (AgentSet) |
| Seeded RNG | `Random.Xoshiro(seed)` | `super().__init__(seed=seed)` |